# PRÁTICA GUIADA - Data Wrangling.

### PARTE I: Limpeza e transformação de dados.

#### Esta prática tem como objetivo fornecer um catálogo de [métodos](https://towardsdatascience.com/tagged/data-wrangling) e funções em Pandas e Python que podem ser úteis ao lidar com tarefas de limpeza de dados. 

#### Em geral, podemos identificar seis tipos de tarefas ou operações que são aplicadas aos dados no estágio de limpeza.

1. Padronização de categorias (homogeneização).
2. Resolução de problemas de formato.
3. Atribuição de formato adequados (dtype).
4. Correção de valores incorretos.
5. Preencher dados faltantes (missing data imputation).
6. Organização correta do conjunto de dados (tidy data).

#### As funções e métodos apresentados abrangem uma ou várias dessas operações.

### Remover duplicados

In [1]:
import pandas as pd
import numpy as np

In [11]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4, 
                     'k2': [1, 1, 2, 3, 3, 3, 4]
                    }
                   )
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,3
6,two,4


#### O método  [`.duplicated()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) retorna um booleano identificando os casos duplicados.

In [6]:
print('Registros duplicados:', data.duplicated().sum() )
data.duplicated()

Registros duplicados: 3


0    False
1     True
2    False
3    False
4     True
5     True
6    False
dtype: bool

#### Já o método [`drop_duplicates()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) retorna o `DataFrame` sem os casos [duplicados](https://medium.com/swlh/removing-duplicates-in-python-26bc788768eb).

In [7]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
6,two,4


In [13]:
#print(data.duplicated())
print(~data.duplicated())
data[~data.duplicated()] == data.drop_duplicates()

0     True
1    False
2     True
3     True
4    False
5    False
6     True
dtype: bool


,k1,k2
0,True,True
2,True,True
3,True,True
6,True,True


#### Pode ser utilizado [`drop_duplicates()`](https://www.xspdf.com/resolution/51637141.html) para eliminar duplicados em uma só coluna ou em um conjunto de colunas.

In [14]:
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,3
6,two,4


In [15]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,3,5
6,two,4,6


In [16]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


In [17]:
data.drop_duplicates(['k1', 'k2'])

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
6,two,4,6


### Mapear e transformar os dados.

#### É posível [mapear](https://dzone.com/articles/know-how-data-mapping-supports-data-transformation) os dadis a partir de um dicionário, é possível criar uma nova coluna para um Dataframe, onde as chaves são vinculadas a uma das séries e os valores fazem parte da nova coluna.

In [18]:
data = pd.DataFrame({'food': ['bacon', 
                              'pulled pork', 
                              'bacon', 
                              'Pastrami', 
                              'corned beef', 
                              'Bacon', 
                              'pastrami', 
                              'honey ham', 
                              'nova lox'], 
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]
                    }
                   )
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


#### Podemos usar o método [`.unique()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.unique.html?highlight=unique#pandas.unique) para retornar dados por ordem de aparição.

In [20]:
#data.food.unique()
data['food'].unique()

array(['bacon', 'pulled pork', 'Pastrami', 'corned beef', 'Bacon',
       'pastrami', 'honey ham', 'nova lox'], dtype=object)

#### A ideia agora é atribuir um `animal` a cada tipo de `carne`. 

In [22]:
meat_to_animal = { 
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

#### Uma opção é fazer isso com os métodos [`.map()`](https://docs.python.org/3/library/functions.html#map), ela [retorna](https://towardsdatascience.com/understand-map-function-to-manipulate-pandas-series-8ac340d514f7) um iterador que aplica uma função a [cada](https://towardsdatascience.com/using-map-and-filter-in-python-ffdfa8b97520) item do iteravel, provendo o resultado.

In [23]:
pd.DataFrame(meat_to_animal.values(), 
             index = meat_to_animal.keys()
            ).reset_index().rename({0:'animal'}, 
                                   axis = 1 
                                  )

,index,animal
0,bacon,pig
1,pulled pork,pig
2,pastrami,cow
3,corned beef,cow
4,honey ham,pig
5,nova lox,salmon


In [24]:
data['animal'] = data['food'].apply(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### Substituir valores

#### O método [`.replace()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html) oferece várias maneiras de fazer substituições em uma série de Pandas: 

    1- Um valor antigo por um novo valor;
    2- Uma lista de valores antigos por um novo valor; 
    3- Uma lista de valores antigos por uma lista de valores novos; 
    4- Um dicionário que mapeia valores novos e antigos.

In [27]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [28]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [29]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

#### Usando um [dicionário](http://pythoninthewyld.com/2018/03/12/dict-based-find-and-replace-deluxe/). 

In [30]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Discretizar e binarizar variáveis

#### O processo de transformar uma variável numérica em categórica se chama discretização. O método [`.cut()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html?highlight=cut#pandas.cut) retorna o intervalo semifechado ao qual cada entrada pertence.

In [31]:
ages = [26, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

data_2 = pd.DataFrame(ages, 
                      columns = ['ages']
                     )
data_2

,ages
0,26
1,22
2,25
3,27
4,21
5,23
6,37
7,31
8,61
9,45


#### Vamos definir os valores de corte e obter uma lista de intervalos.

In [33]:
bins = [18, 25, 35, 60, 100]

data_2['cats'] = pd.cut(data_2['ages'], bins)
data_2

,ages,cats
0,26,"(25, 35]"
1,22,"(18, 25]"
2,25,"(18, 25]"
3,27,"(25, 35]"
4,21,"(18, 25]"
5,23,"(18, 25]"
6,37,"(35, 60]"
7,31,"(25, 35]"
8,61,"(60, 100]"
9,45,"(35, 60]"


#### O númeor de elementos por agrupamento.

In [35]:
data_2['cats'].value_counts()

(25, 35]     4
(18, 25]     4
(35, 60]     3
(60, 100]    1
Name: cats, dtype: int64

#### Podemos modificar a inclusão do valor de corte nos intervalos.

In [36]:
pd.cut(ages, [18, 26, 36, 61, 100], 
       right = False
      )

[[26, 36), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

#### Podemos atribuir tags às categorias.

In [37]:
group_names = ['Youth', 
               'YoungAdult', 
               'MiddleAged', 
               'Senior'
              ]

data_2['cats'] = pd.cut(data_2['ages'], 
                        bins, 
                        labels = group_names
                       )

data_2

,ages,cats
0,26,YoungAdult
1,22,Youth
2,25,Youth
3,27,YoungAdult
4,21,Youth
5,23,Youth
6,37,MiddleAged
7,31,YoungAdult
8,61,Senior
9,45,MiddleAged


### Quantis em vez de intervalos preestabelecidos.

#### Vamos dividir em quantis, nesse caso $10$, usando o método [.qcut()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html), que aplica uma função de discretização baseada em [quantis](https://en.wikipedia.org/wiki/Quantile).

In [39]:
data = np.random.randn(1000)
cats = pd.qcut(data, 5) 
cats

[(0.878, 2.978], (-0.815, -0.278], (-2.972, -0.815], (0.285, 0.878], (-2.972, -0.815], ..., (0.285, 0.878], (-2.972, -0.815], (-2.972, -0.815], (-0.815, -0.278], (-0.278, 0.285]]
Length: 1000
Categories (5, interval[float64]): [(-2.972, -0.815] < (-0.815, -0.278] < (-0.278, 0.285] < (0.285, 0.878] < (0.878, 2.978]]

In [40]:
pd.value_counts(cats).sort_index()

(-2.972, -0.815]    200
(-0.815, -0.278]    200
(-0.278, 0.285]     200
(0.285, 0.878]      200
(0.878, 2.978]      200
dtype: int64

### Detectar e filtrar outliers

#### A definição padrão de [`outlier`](https://medium.com/towards-artificial-intelligence/outlier-detection-and-treatment-a-beginners-guide-c44af0699754) determina que outliers são todos os valores que estão a mais de 3 desvios padrão acima ou abaixo da média.

#### Obs: sempre que utilizar alguma função baseada em aleatoriedade (como no exemplo abaixo), garanta que a semente `seed` está definida para algum valor fixo. Isso significa que o código irá gerar sempre o mesmo conjunto de valores aleatórios, permitindo que sua pesquisa apresente os mesmos resultados em outros momentos ou computadores.

In [42]:
np.random.seed(123)
data_3 = pd.DataFrame(np.random.randn(1000, 4))
data_3

,0,1,2,3
0,-1.085631,0.997345,0.282978,-1.506295
1,-0.578600,1.651437,-2.426679,-0.428913
2,1.265936,-0.866740,-0.678886,-0.094709
3,1.491390,-0.638902,-0.443982,-0.434351
4,2.205930,2.186786,1.004054,0.386186
...,...,...,...,...
995,-0.499897,0.587647,-0.926542,1.736982
996,-0.459550,0.125822,-1.119947,-0.521887
997,-2.013430,-0.028708,-0.103142,-1.761313
998,-0.185167,0.504077,1.354567,-0.907952


In [43]:
data_3.sample(5)

,0,1,2,3
468,0.181974,0.572843,-0.839113,0.192449
373,0.997890,1.686037,-0.794252,0.215802
280,1.655773,0.481287,-0.310228,-0.552144
764,-0.721146,0.389152,0.805637,1.407747
369,1.836869,-0.718403,1.134945,1.549722


In [44]:
np.random.seed(123)
data_3.sample(5)

,0,1,2,3
131,-0.448392,0.412819,0.600883,-1.131641
203,-1.415519,1.629611,1.052401,-0.148405
50,0.703310,-0.598105,2.200702,0.688297
585,2.156086,0.275040,-0.174344,-0.714881
138,-0.121741,-1.762898,1.158069,-0.682765


In [45]:
summary = data_3.describe()
summary

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.007502,0.039160,-0.010286,0.024285
std,0.977024,0.973484,1.012230,0.970421
min,-3.167055,-2.920029,-3.801378,-3.231055
25%,-0.662012,-0.636160,-0.687717,-0.599195
50%,-0.024843,0.062549,0.007035,0.038718
75%,0.613950,0.672448,0.664586,0.683228
max,3.050755,2.850708,2.766603,3.571579


In [47]:
mean = summary.loc['mean', 3]
std = summary.loc['std', 3]
threshold_min = mean - 3 * std
threshold_max = mean + 3 * std

In [50]:
print('mean:', round(mean, 2), 
      'std:', round(std, 2), 
      'threshold_min:', round(threshold_min, 2), 
      'threshold_max:', round(threshold_max, 2)
     )

mean: 0.02 std: 0.97 threshold_min: -2.89 threshold_max: 2.94


In [53]:
mask = (data_3[3] > threshold_min) & (data_3[3] < threshold_max)
#print(mask)
data_3.loc[mask, : ]

,0,1,2,3
0,-1.085631,0.997345,0.282978,-1.506295
1,-0.578600,1.651437,-2.426679,-0.428913
2,1.265936,-0.866740,-0.678886,-0.094709
3,1.491390,-0.638902,-0.443982,-0.434351
4,2.205930,2.186786,1.004054,0.386186
...,...,...,...,...
995,-0.499897,0.587647,-0.926542,1.736982
996,-0.459550,0.125822,-1.119947,-0.521887
997,-2.013430,-0.028708,-0.103142,-1.761313
998,-0.185167,0.504077,1.354567,-0.907952


####  <span style = "color:blue">Prática independente.</span>

#### Mostre as linhas que possuem outliers na coluna $3$.

####  <span style = "color:red">Código original.</span>
<!---
mask_inverse = ~((data_3[3] > threshold_min) & (data_3[3] < threshold_max))
data_3.loc[mask_inverse,: ]
-->

#### Mostre as linhas que possuem `outliers` em pelo menos uma das colunas.

####  <span style = "color:red">Código original.</span>
<!---
data_3.loc[data_3.apply(lambda col: (col[np.abs(col) > np.mean(col) + 3 * np.std(col)]).notnull()).fillna(False).index, : ]
-->

### PARTE II: Variáveis categóricas e Dummies


#### O uso de variáveis dummies, também conhecido como ["one hot encoding"](https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd), pode ser interpretado como o processo inverso da discretização. 

#### Nesse caso, pegamos variáveis categóricas e as transformamos em variáveis numéricas que seguem uma [distribuição binomial](https://www.statisticshowto.com/probability-and-statistics/binomial-theorem/binomial-distribution-formula/) com (probabilidade](https://www.statisticshowto.com/probability-and-statistics/probability-main-index/) `p`, em que `p` é a quantidade de vezes que a categoria aparece sobre o total de dados.

In [61]:
df = pd.DataFrame({'cat_produto': ['b', 'b', 'a', 'c', 'a', 'b'], 
                   'cod_venda': np.arange(100, 112, 2)}
                 )
df

,cat_produto,cod_venda
0,b,100
1,b,102
2,a,104
3,c,106
4,a,108
5,b,110


#### Pandas conta com o método [`.get_dummies()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) que recebe uma Série ou uma lista de Séries e realiza o `one hot encoding``.

#### Lembre que uma variável com `k` categorias pode ser representada por `k-1` variáveis. Dessa forma, um dos parâmetros-chave do método `.get_dummies()` é `drop_first`, que gera `k-1` categorias, quando definido como `TRUE`.

In [62]:
pd.get_dummies(df['cat_produto'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


#### Adicionamos um prefixo para identificar a categoria

In [64]:
dummies = pd.get_dummies(df['cat_produto'], 
                         prefix = 'cat_produto',
                         drop_first = True
                        )

dummies

,cat_produto_b,cat_produto_c
0,1,0
1,1,0
2,0,0
3,0,1
4,0,0
5,1,0


#### Concatenamos a coluna `cod_venda`.

In [65]:
df_with_dummy = df.join(dummies)
df_with_dummy

,cat_produto,cod_venda,cat_produto_b,cat_produto_c
0,b,100,1,0
1,b,102,1,0
2,a,104,0,0
3,c,106,0,1
4,a,108,0,0
5,b,110,1,0


## Manipulação de strings

### String object methods

#### O método [`.split()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html?highlight=split#pandas.Series.str.split) pega uma string, divide-a de acordo com um delimitador ([`sep`](https://realpython.com/python-string-split-concatenate-join/)) e retorna uma lista.

In [88]:
val = 'a, b, guido, asjd, kle'

val.split(',')

['a', ' b', ' guido', ' asjd', ' kle']

#### o método [`.strip()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.strip.html?highlight=strip#pandas.Series.str.strip) pega uma string e retorna uma string sem os espaços iniciais e finais.

In [89]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido', 'asjd', 'kle']

In [90]:
val

'a, b, guido, asjd, kle'

#### Repare no [operador](https://www.geeksforgeeks.org/python-membership-identity-operators-not-not/) [`in`](https://docs.python.org/3.4/library/operator.html).

In [91]:
'guido' in val

True

#### O método [`.find()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.find.html?highlight=find#pandas.Series.str.find) retorna o menor índice dentro de uma `string` na qual uma `substring` se encontra. Se não a encontra, retorna `-1`.

In [92]:
val.find(':')
#val.find('guido')

-1

#### O método [`.index ()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.html?highlight=index#pandas.Index) é semelhante, mas retorna um `ValueError` quando a substring procurada não é encontrada

In [93]:
val.index(':')
#val.index('guido')

ValueError: substring not found

#### O mesmo exemplo acima, mas usando [exceções](https://realpython.com/python-exceptions/), permite-nos lidar com erros no tempo de execução.

In [99]:
try:
    val.index(':')
#    val.index('guido')    
    print("Tudo ok!")    
except ValueError:
    print("Erro, substring não encontrada!")

Erro, substring não encontrada!


#### O método [`.count()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.count.html?highlight=count#pandas.DataFrame.count) conta a ocorrência de uma `substring` determinada em uma string maior.

In [100]:
val.count(',')

4

#### O método [`.replace()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html?highlight=replace#pandas.DataFrame.replace) substitui uma `substring` por outra.

In [101]:
val.replace(',', ';')

'a; b; guido; asjd; kle'

####  <span style = "color:red">Parei Aqui.</span>

### Funções vetorizadas para strings em Pandas.

#### Vamos a seguir definir um dicionário com nomes e emails e alguns padrões de busca.

In [106]:
import re
data = {'Dave': 'dave@google.com', 
        'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com'}

data = pd.Series(data)

In [107]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
dtype: object

In [113]:
pattern_1 = 'steve'
pattern_2 = ['steve', 'dave']
pattern_3 = '\w'
pattern_4 = '\w+'

#### O método [`.findall()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.findall.html?highlight=findall#pandas.Series.str.findall) encontra todas as ocorrências de padrão ou [expressões regulares](https://docs.python.org/pt-br/3.8/howto/regex.html) em um objeto `Series`/`Index`.

In [117]:
data.str.findall(pattern_1)

Dave          []
Steve    [steve]
Rob           []
dtype: object

In [118]:
data.str.findall(pattern_2)

TypeError: unhashable type: 'list'

In [119]:
data.str.findall(pattern_3)

Dave     [d, a, v, e, g, o, o, g, l, e, c, o, m]
Steve    [s, t, e, v, e, g, m, a, i, l, c, o, m]
Rob            [r, o, b, g, m, a, i, l, c, o, m]
dtype: object

In [120]:
data.str.findall(pattern_4)

Dave     [dave, google, com]
Steve    [steve, gmail, com]
Rob        [rob, gmail, com]
dtype: object

#### O método [`.str.match()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.match.html?highlight=str%20match#pandas.Series.str.match) determina se cada string coincide com uma expressão regular.

In [121]:
data[data.str.match(pattern_1)]

Steve    steve@gmail.com
dtype: object

In [124]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
dtype: object

### Exemplo: Dataset movies

In [125]:
mnames = ['movie_id', 
          'title', 
          'genres'
         ]

movies = pd.read_table('movies.csv', 
                       header = None, 
                       names = mnames, 
                       encoding = "latin9", 
                       sep = ';'
                      )
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


#### Subdividimos os gêneros.

In [130]:
#movie_genres_split = movies.genres.str.split('|').values
movie_genres_split = movies['genres'].str.split('|').values
movie_genres_split

array([list(['Animation', "Children's", 'Comedy']),
       list(['Adventure', "Children's", 'Fantasy']),
       list(['Comedy', 'Romance']), ..., list(['Drama']), list(['Drama']),
       list(['Drama', 'Thriller'])], dtype=object)

#### Criando uma lista de gêneros.

In [135]:
genres = set([item for s in movie_genres_split for item in s])
genres

{'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

#### Criamos e codificam os as categorias como dummies. Escreve um $1$ onde for correspondente.

In [138]:
dummies = pd.DataFrame(np.zeros((len(movies), 
                                 len(genres))
                                , dtype = int
                               ), 
                       columns = genres
                      )
dummies.head(10)

,Fantasy,Drama,Crime,Action,Children's,War,Animation,Adventure,Romance,Horror,Documentary,Sci-Fi,Musical,Mystery,Comedy,Film-Noir,Western,Thriller
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### A função [`enumerate()`](https://docs.python.org/3/library/functions.html#enumerate), retorna um objeto enumerado, em queo  argumento iterável deve ser uma sequência, um iterador ou algum outro objeto que suporte iteração.

In [139]:
for i, gen in enumerate(movies.genres):
    if i < 10:
        print(i, gen)
    dummies.loc[i, gen.split('|')] = 1

0 Animation|Children's|Comedy
1 Adventure|Children's|Fantasy
2 Comedy|Romance
3 Comedy|Drama
4 Comedy
5 Action|Crime|Thriller
6 Comedy|Romance
7 Adventure|Children's
8 Action
9 Action|Adventure|Thriller


In [140]:
dummies.head()

,Fantasy,Drama,Crime,Action,Children's,War,Animation,Adventure,Romance,Horror,Documentary,Sci-Fi,Musical,Mystery,Comedy,Film-Noir,Western,Thriller
0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


#### O método [`.join()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html), junta colunas com outro `DataFrame` no índice ou em uma coluna-chave.

In [143]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.head()

,movie_id,title,genres,Genre_Fantasy,Genre_Drama,Genre_Crime,Genre_Action,Genre_Children's,Genre_War,Genre_Animation,...,Genre_Romance,Genre_Horror,Genre_Documentary,Genre_Sci-Fi,Genre_Musical,Genre_Mystery,Genre_Comedy,Genre_Film-Noir,Genre_Western,Genre_Thriller
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
